<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from AlazarPowerSweepData import *

%matplotlib qt

#project_path = r"G:\Shared drives\Quasiparticles_2\NBR07_Oct2022\test_power_sweep" 
project_path = str(input(r"Path to Measurement Run Root: "))

power_sweep_obj = AlazarPowerSweepData(project_path)

Path to Measurement Run Root: G:\Shared drives\Quasiparticles_2\NBR07_Oct2022\test_power_sweep


In [2]:
createPdf = False

In [3]:
power_sweep_obj.process_Alazar_Data(avgTime=2,plots=createPdf)

Creating figure paths.....
Reading and sorting data files.....
Data loaded without creating IQ downsampled plots....


In [4]:
intTime=1
SNRmin=3

In [ ]:
power_sweep_obj.start_HMM_fit(intTime=intTime, SNRmin= SNRmin)



==========	HMM ANALYSIS STARTED	==========

Attenuation below which the system goes non-linear: 4
Please Ensure the Following Attenuation Configuration is Correct:


LO_power = 20
Splitter = -3
Fixed_attenuator_1 = -20
Fixed_attenuator_2 = -30
DA_fixed = -10
Fridge_attenuation = -75
Ecosorb = -3
Press Y/y if Correct or N/n if not: y
Please Ensure the Following MetaData is Correct:


dateOfMeasurement = 20221005
Temp = 30
eccosorb = True
KandL = True
rigidCables = True
JPA = True
TWPA = True
ClearingCoupler = False
sampleRateMHz = 10
durationSeconds = 5
f0 = 4.2727
LOf = 4.27238
phi = 0.47
Press Y/y if Correct or N/n if not: y

The power to the device is -125 dBM at the chosen attenuation 4

Number of Modes you want to fit: 3
Extracted Means: [(-33.58759023581993, -21.066262355718223), (13.617003684825079, 15.716538101927227), (35.318855954835925, -36.269819878211685)]
Gussed Covariance:
[[[5 1]
  [1 5]]

 [[5 1]
  [1 5]]

 [[5 1]
  [1 5]]]


Starting HMM Analysis.....


HMM done on a